In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# This header make the program uses proper number of GPUs.
# If not set this header, the program will use all GPUs.

import numpy as np
import tensorflow as tf
import sys
from pandas import read_csv
import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.models import model_from_json
import matplotlib.pyplot as plt
from IPython.display import clear_output
np.random.seed(123)  # for reproducibility

# Extra part for memory management
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
tf.Session(config = config)

config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)

Using TensorFlow backend.


In [6]:
def Test_LSTM(state_condition, structure, peak_hour, pred_hour, LSTM_hour, 
                num_unit, num_epoch, train_index, val_index, test_index, optmzr, loss_ftn):
    #file_condition = '1hPEAK_6hLSTM_log'
    
    len_PRED = pred_hour/peak_hour
    len_LSTM = LSTM_hour/peak_hour
    file_condition = 'SineTest_'+str(peak_hour)+'hPEAK_'+str(pred_hour)+'hPrediction_'+str(LSTM_hour)+'hLSTM'
    
    if len_PRED <= 0:
        print("You entered wrong predction hour or peak hour")
        sys.eixt()
    if len_LSTM <= 0:
        print("You entered wrong LSTM hour or peak hour")
        sys.eixt()
        
    len_PRED = int(len_PRED)
    len_LSTM = int(len_LSTM)
       
    allDataSet = read_csv('C:\\Users\\YKW\\Desktop\\SEP_TEMP\\'+file_condition+'.csv', sep=',')
    allDataSet_size = len(allDataSet)
    
    print("allDataSet: ", allDataSet.shape)
    print("allDataSet_size: ", allDataSet_size)
    print("allDataSet Type: ", type(allDataSet))
    print("allDataSet_size Type: ", type(allDataSet_size))
    
    data_count = np.arange(len_LSTM)+1
    data_count = data_count* (-1)
    data_count = data_count +len_LSTM+1
    data_title = []
    
    for bi in range(len(data_count)):
        data_title.append('b'+str(int(data_count[bi])))
        
    pred_count = np.arange(len_PRED)
    pred_title = []
    for ai in range(len(pred_count)):
        pred_title.append('a'+str(int(pred_count[ai])))

    flux = allDataSet[data_title].values
    now_flux = allDataSet['a0'].values
     
    d_flux = np.array([flux])
    d_now_flux = np.array([now_flux])
    
    d_flux = np.array([d_flux]).reshape(allDataSet_size, len_LSTM,1) ### d_flux = np.array(d_flux).reshape(allDataSet_size, 1, len_LSTM)
    d_now_flux = np.array([d_now_flux]).reshape(allDataSet_size, 1)
    

    def M_LSTM_Stateful_Inspace(shapes):
        model = Sequential()
        model.add(LSTM(num_unit,batch_input_shape=shapes, stateful=True))
        model.add(Dense(1))
        model.summary()
        return model
            
    def M_LSTM_Stateless_Inspace(shapes):
        model = Sequential()
        model.add(LSTM(num_unit,input_shape=shapes, return_sequences=True, stateful=False))
        model.add(Dropout(0.3))
        model.add(LSTM(num_unit,input_shape=shapes, return_sequences=True, stateful=False))
        model.add(Dropout(0.3))
        model.add(LSTM(num_unit,input_shape=shapes, stateful=False))
        model.add(Dense(1))
        model.summary()
        return model
    
    def M_LSTM_Stateful_SeaT(shapes):
        model = Sequential()
        model.add(LSTM(num_unit,batch_input_shape=shapes, stateful=True))
        model.add(Dense(7))
        model.add(Dense(1))
        model.summary()
        return model
            
    def M_LSTM_Stateless_SeaT(shapes):
        model = Sequential()
        model.add(LSTM(num_unit,input_shape=shapes, stateful=False))
        model.add(Dense(7))
        model.add(Dense(1))
        model.summary()
        return model


    class PlotLosses(keras.callbacks.Callback):
        def on_train_begin(self, logs={}):
            self.i = 0
            self.x = []
            self.losses = []
            self.val_losses = []

            self.fig = plt.figure()

            self.logs = []

        def on_epoch_end(self, epoch, logs={}):

            self.logs.append(logs)
            self.x.append(self.i)
            self.losses.append(logs.get('loss'))
            self.val_losses.append(logs.get('val_loss'))
            self.i += 1
            clear_output(wait=True)
            plt.plot(self.x, self.losses, label="loss")
            plt.plot(self.x, self.val_losses, label="val_loss")
            plt.legend()
            plt.show();
            

    if structure is 'Inspace':
        weightfileName = 'C:\\Users\\YKW\\Desktop\\SEP_TEMP\\ReturnSQ_'+state_condition+'_StepUpOutput_LSTM_GOES15_pgt10_'+file_condition+'_ep'+str(num_epoch)+'_'+str(num_unit)+'units_'+optmzr+'_' + loss_ftn + '_MODEL_WEIGHT'
        testfileName = 'C:\\Users\\YKW\\Desktop\\SEP_TEMP\\ReturnSQ_'+state_condition+'_StepUpOutput_TEST_'+file_condition+'_ep'+str(num_epoch)+'_'+str(num_unit)+'units_'+optmzr + '_' + loss_ftn + '_Proton Flux Prediction By LSTM Model Train.csv'
    else:
        if structure is 'SeaT':
            weightfileName = 'C:\\Users\\YKW\\Desktop\\SEP_TEMP\\ReturnSQ_'+state_condition+'_StepUpOutput_LSTM_GOES15_pgt10_'+file_condition+'_ep'+str(num_epoch)+'_'+str(num_unit)+'units_'+optmzr+'_' + loss_ftn + '_MODEL_WEIGHT'
            testfileName = 'C:\\Users\\YKW\\Desktop\\SEP_TEMP\\ReturnSQ_'+state_condition+'_StepUpOutput_Dense7_TEST_'+file_condition+'_ep'+str(num_epoch)+'_'+str(num_unit)+'units_'+optmzr + '_' + loss_ftn + '_Proton Flux Prediction By LSTM Model Train.csv'
        else:
            print("You entered wrong ")
            sys.exit()
            

    plot_losses = PlotLosses()
    
    if state_condition is 'Stateful':
        inputFluxShape = (1,1,len_LSTM)
        if structure is 'Inspace':
            model = M_LSTM_Stateful_Inspace(inputFluxShape)
        if structure is 'SeaT':
            model = M_LSTM_Stateful_SeaT(inputFluxShape)
    else:
        if state_condition is 'Stateless':
            inputFluxShape = (len_LSTM,1) ### inputFluxShape = (1,len_LSTM)
            if structure is 'Inspace':
                model = M_LSTM_Stateless_Inspace(inputFluxShape)
            if structure is 'SeaT':
                model = M_LSTM_Stateless_SeaT(inputFluxShape)
        else:
            print("You enterd wrong state_condition!")
            sys.exit()
 

    val_index  = train_index + val_index
    test_index = val_index + test_index
    
    trainX = d_flux[:train_index]
    trainY = d_now_flux[:train_index]
        
    valX = d_flux[train_index:val_index]
    valY = d_now_flux[train_index:val_index]
        
    testX = d_flux[val_index:test_index]
    testY = d_now_flux[val_index:test_index]
    
    model.compile(loss=loss_ftn, optimizer=optmzr, metrics=['acc'])
     
    if state_condition is 'Stateful':    
        for epoch_idx in range(num_epoch):
            model.fit(trainX, trainY, epochs=1, batch_size=1, verbose=2, shuffle=False, validation_data=(valX, valY))
            model.reset_states()
            print(epoch_idx)
    else:
        if state_condition is 'Stateless': 
            model.fit(trainX, trainY, epochs=num_epoch, batch_size=1400, verbose=2, validation_data=(valX, valY))

        
    model_json = model.to_json()
    with open(weightfileName + ".json", "w") as json_file:
        json_file.write(model_json)
    model.save_weights(weightfileName + ".h5")
    print("Saved model to disk")

    json_file = open(weightfileName + '.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)

    loaded_model.load_weights(weightfileName + ".h5")
    print("Loaded model from disk")
    loaded_model.compile(loss=loss_ftn, optimizer=optmzr, metrics=['acc'])
    

    #date_testY = allDataSet['date'].values[val_index:test_index]
    f = open(testfileName, 'w')
    f.write('Ground')
    for i in range(len_PRED):
        f.write(','+ 'a'+str(i))
    f.write('\n')
    
        
    for idx2 in range(0, len(testX)):
        
        NowY = testY[idx2:idx2+1]
            
        pred = []
        pred_in = testX[idx2:idx2+1]

        for pred_n in range(len_PRED):
            resultY = model.predict(pred_in)
            pred.append(resultY)
        
            pred_in = np.append(pred_in,resultY)
            pred_in = pred_in[1:]
            pred_in = pred_in.reshape(1, len_LSTM, 1)

        f.write(str(NowY[0][0]))
            
        for nwr in range(len_PRED):
            f.write(',' + str(pred[nwr]))   
            
        f.write('\n')
        pred_in = []
        resultY = []
        print(idx2)
         
    f.close()


###################################################################################################

In [3]:
num_epoch = 400
num_unit = 240

Temp_LSTM('Stateless', 'Inspace', 1, 24, 48, num_unit, num_epoch, 2520, 360, 720, "Adam",'mean_squared_error')

allDataSet:  (3600, 72)
allDataSet_size:  3600
allDataSet Type:  <class 'pandas.core.frame.DataFrame'>
allDataSet_size Type:  <class 'int'>
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 48, 240)           232320    
_________________________________________________________________
dropout_1 (Dropout)          (None, 48, 240)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 48, 240)           461760    
_________________________________________________________________
dropout_2 (Dropout)          (None, 48, 240)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 240)               461760    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 241       
To